<a href="https://colab.research.google.com/github/nickwan/nwds-stream-notebooks/blob/main/cudf_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cudf-cu12 --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.4/582.4 kB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
  Attempting uninstall: protobuf
    Fou

In [ ]:
import pandas as pd
import os

In [ ]:
%%timeit
project_dir = '/kaggle/input/nfl-big-data-bowl-2024'
players = pd.read_csv(f'{project_dir}/players.csv')
games = pd.read_csv(f'{project_dir}/games.csv')
tackles = pd.read_csv(f'{project_dir}/tackles.csv')
plays = pd.read_csv(f'{project_dir}/plays.csv')
week = pd.read_csv(f'{project_dir}/tracking_week_1.csv')

tackles['pressure'] = tackles.loc[:, ['tackle', 'assist', 'forcedFumble', 'pff_missedTackle']].sum(axis=1)
week = week.merge(tackles.loc[:, ['gameId', 'playId','nflId','pressure']], how='left')
week['pressure'] = week['pressure'].fillna(0).astype(int)

games['scoreDiff'] = games['homeFinalScore'].sub(games['visitorFinalScore'])
cols = ['gameId', 'season', 'week', 'gameTimeEastern',
        'homeFinalScore', 'visitorFinalScore', 'scoreDiff']
week = week.merge(games.loc[:, cols], how='left')

week = week.merge(players.loc[:, ['nflId', 'collegeName', 'position']], how='left')
plays['ballCarrier'] = 1
week = week.merge(plays.loc[:, ['gameId', 'playId', 'ballCarrierId', 'ballCarrier']].rename(columns={'ballCarrierId':'nflId'}), how='left')
week['ballCarrier'] = week['ballCarrier'].fillna(0)

cols = ['gameId', 'playId', 'quarter', 'down', 'yardsToGo',
        'possessionTeam', 'defensiveTeam','homeTeamWinProbabilityAdded',
        'visitorTeamWinProbilityAdded', 'expectedPoints','expectedPointsAdded']
week = week.merge(plays.loc[:, cols], how='left')


In [ ]:
# This line is not needed, this is just to
# restart the notebook without clearing the outputs
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [ ]:
%load_ext cudf.pandas

/usr/local/lib/python3.10/dist-packages/cupy/_environment.py:447: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


ImportError: ignored

In [ ]:
import pandas as pd
import os

In [ ]:
%%timeit
project_dir = '/kaggle/input/nfl-big-data-bowl-2024'
players = pd.read_csv(f'{project_dir}/players.csv')
games = pd.read_csv(f'{project_dir}/games.csv')
tackles = pd.read_csv(f'{project_dir}/tackles.csv')
plays = pd.read_csv(f'{project_dir}/plays.csv')
week = pd.read_csv(f'{project_dir}/tracking_week_1.csv')

tackles['pressure'] = tackles.loc[:, ['tackle', 'assist', 'forcedFumble', 'pff_missedTackle']].sum(axis=1)
week = week.merge(tackles.loc[:, ['gameId', 'playId','nflId','pressure']], how='left')
week['pressure'] = week['pressure'].fillna(0).astype(int)

games['scoreDiff'] = games['homeFinalScore'].sub(games['visitorFinalScore'])
cols = ['gameId', 'season', 'week', 'gameTimeEastern',
        'homeFinalScore', 'visitorFinalScore', 'scoreDiff']
week = week.merge(games.loc[:, cols], how='left')

week = week.merge(players.loc[:, ['nflId', 'collegeName', 'position']], how='left')
plays['ballCarrier'] = 1
week = week.merge(plays.loc[:, ['gameId', 'playId', 'ballCarrierId', 'ballCarrier']].rename(columns={'ballCarrierId':'nflId'}), how='left')
week['ballCarrier'] = week['ballCarrier'].fillna(0)

cols = ['gameId', 'playId', 'quarter', 'down', 'yardsToGo',
        'possessionTeam', 'defensiveTeam','homeTeamWinProbabilityAdded',
        'visitorTeamWinProbilityAdded', 'expectedPoints','expectedPointsAdded']
week = week.merge(plays.loc[:, cols], how='left')
